# ファイルシステム周りのシステムコールとGoの関数について

2018/9/26 勉強会

## syscall.inotifyについて

- inotify


### inotify

- ideno notifyの略
- ファイルシステムへの変更を通知する
- dnotifyの派生
    - ディレクトリレベルでの監視しかできない
- Linux 2.6.13カーネルから組み込まれた(2005/8/29リリース)
- 1つのファイルディスクリプタを開くだけで1つ以上のファイルやディレクトリでの以下のことを監視できる
    - Open
    - Close
    - 移動/名前変更
    - 削除
    - 作成
    - 属性の変更
    - etc.

- inotifyコマンド
    - http://www.usupi.org/sysad/157.html
    
    
【参考】
- [inotify で Linux ファイルシステムのイベントを監視する](https://www.ibm.com/developerworks/jp/linux/library/l-inotify/index.html)

## 実践

- [go_fsnotify.go](https://github.com/mrsekut/GoSystemProgramming/blob/master/10/go_fsnotify.go)


### fsnotify参考
- [fsnotify/fsnotify: Cross-platform file system notifications for Go.](https://github.com/fsnotify/fsnotify)
- [fsnotify - GoDoc](https://godoc.org/github.com/fsnotify/fsnotify)
- [[Go][Mac] ファイルの変更を検知する方法 - Qiita](https://qiita.com/cotrpepe/items/3877a8d803f45c6f1171)

## syscall.Flock

- fcntl

## ファイルロック

- 排他制御
- 複数のプロセス間で同じリソースを同時に変更しないようにするために、他のプロセスに「このリソース使用してるなう」と伝える手法の一つ　　


### よくある例

- カウンター増えない問題
- 銀行口座の上書き

### syscall.Flock()

- ファイルロックをかけられる
- 後からアクセスしてきたプロセスが確認してくれなければ意味がない
    - アクセスしてくるプロセス全員がFlock()すれば、かかっていれば何もしない、かかってなければかけるといったことができる
    - ロックはかかっているが、無視して書きこむこともできる
- これを勧告ロックという
    - 対義語は強制ロック




### Note


- ノンブロッキングモード
    - 定期的にロックをかけようとトライする
- ブロッキングモード


### 共有ロックと排他ロック

- 共有ロック
    - 読み込むときに使用される
    - 複数のプロセスから同じリソースに対していくつも同時にかけられる
    - 他のプロセスがロックをかけようとした場合は、共有ロックは許可するが、排他ロックを許可しない

- 排他ロック
    - 書きこむときに使用される
    - 他のプロセスからの共有ロックがブロックされる
    - 他のプロセスがロックをかけようとした場合は、共有ロックも排他ロックも許可しない

#### 結局どういうことかというと

- 書き込みは一人のみ、読み込みは複数人でもOKってこと



## 実践

- [go_flock.go](https://github.com/mrsekut/GoSystemProgramming/blob/master/10/go_flock.go)
    - 2つ実行する

## syscall.Mmap()

- mmap


## ファイルのメモリへのマッピング

- syscall.Mmap()
    - メモリマップ
    - ファイルの中身をそのままメモリ上に展開できる
    - メモリ上で書き換えられた内容をそのままファイルに書き込める
    - メモリマップドファイル

## 実践

- [go_mmap.go](https://github.com/mrsekut/GoSystemProgramming/blob/master/10/go_mmap.go)

- mmap.flush()の意味
    - https://stackoverflow.com/questions/20056099/why-does-the-memory-mapped-file-ever-need-to-be-flushed-when-access-is-rdwr

# 参考

- https://www.nttpc.co.jp/yougo/%E3%83%95%E3%82%A1%E3%82%A4%E3%83%AB%E3%82%B7%E3%82%B9%E3%83%86%E3%83%A0.html
- http://blog.matsumoto-r.jp/?p=2030
- http://www.ibm.com/developerworks/linux/library/l-async/
